# Generating Data for Retrieval Augmented Generation. 

In this notebook, we'll generate a dataset for RAG using `sciq` dataset from Huggingface.

In [2]:
!pip install llama-index==0.9.48 datasets tqdm python-dotenv spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 14.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.9 MB/s eta 0:00:00a 0:00:01
Using cached dirtyjson-1.0.8-py3-none-any.whl (25 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 14.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 15.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.1/30.1 MB 10.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 4.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: llama-index
    Found existing installation: llama-index 0.9.39
    Uninstalling llama-index-0.9.39:
      Successfully uninstalled llama-index-0.9.39

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


## Setup Environment Variables

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

## Download sciq dataset

Note: Parts of this repo are inspired by [this](https://srk.ai/blog/004-ai-llm-retrieval-eval-llamaindex) blog on evaluating the performance of RAGs.

In [4]:
from datasets import load_dataset

dataset = load_dataset("sciq")

/Users/rahulparundekar/workspaces/course-openai-api/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Generate RAG Dataset From the Documents

In [5]:
# Let's see what's in the dataset.

dataset["train"].to_pandas().head()

,question,distractor3,distractor1,distractor2,correct_answer,support
0,What type of organism is commonly used in prep...,viruses,protozoa,gymnosperms,mesophilic organisms,"Mesophiles grow best in moderate temperature, ..."
1,What phenomenon makes global winds blow northe...,tropical effect,muon effect,centrifugal effect,coriolis effect,Without Coriolis Effect the global winds would...
2,Changes from a less-ordered state to a more-or...,endothermic,unbalanced,reactive,exothermic,Summary Changes of state are examples of phase...
3,What is the least dangerous radioactive decay?,zeta decay,beta decay,gamma decay,alpha decay,All radioactive decay is dangerous to living t...
4,Kilauea in hawaii is the world’s most continuo...,magma,greenhouse gases,carbon and smog,smoke and ash,Example 3.5 Calculating Projectile Motion: Hot...


We have a `question`, and `support` fields that are super relevant. The answer is also present in the `correct_answer` field, but is one worded.

In [6]:
def cleanup_cols(row):
    return {"question": row["question"], "answer": row["correct_answer"].title(), "support": row["support"]}


updated_dataset = dataset.map(cleanup_cols)

In [7]:
updated_dataset = updated_dataset.remove_columns(["distractor3", "distractor1", "distractor2", "correct_answer"])
updated_dataset["train"].to_pandas().head()

,question,support,answer
0,What type of organism is commonly used in prep...,"Mesophiles grow best in moderate temperature, ...",Mesophilic Organisms
1,What phenomenon makes global winds blow northe...,Without Coriolis Effect the global winds would...,Coriolis Effect
2,Changes from a less-ordered state to a more-or...,Summary Changes of state are examples of phase...,Exothermic
3,What is the least dangerous radioactive decay?,All radioactive decay is dangerous to living t...,Alpha Decay
4,Kilauea in hawaii is the world’s most continuo...,Example 3.5 Calculating Projectile Motion: Hot...,Smoke And Ash


### Save the Dataset in HuggingFace format.

In [8]:
import os

content_folder = os.path.join(os.path.abspath(""), ".content")
print(f"Dataset contains {len(updated_dataset)} rows")
updated_dataset.save_to_disk(content_folder + "/rag_sciq_data.hf")

Dataset contains 3 rows


Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 114890.40 examples/s]


Great! Now we're ready to build our RAG.

In [9]:
updated_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'support', 'answer'],
        num_rows: 11679
    })
    validation: Dataset({
        features: ['question', 'support', 'answer'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'support', 'answer'],
        num_rows: 1000
    })
})